In [1]:
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from transformers import TFT5ForConditionalGeneration, T5Tokenizer
import tensorflow as tf
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# @title Default title text
data = pd.read_csv(r"C:\Users\jasmi\Projects\Final Project\PoliceTraining-Final\training_dataset.csv")

In [4]:
texts = data["Bodycam Transcript"].tolist()
labels = data["Success/Failure"].tolist()

In [5]:
valid_mask = data["Success/Failure"].isin([0, 1])
inputs = data[valid_mask]["Bodycam Transcript"].tolist()
labels = data[valid_mask]["Success/Failure"].tolist()

In [6]:
# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    inputs, labels, test_size=0.2, random_state=42
)

In [7]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenize training and validation data
max_length = 512  # Adjust based on model or dataset requirements
train_encodings = tokenizer(
    train_texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf"
)
val_encodings = tokenizer(
    val_texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="tf"
)



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
def shift_right(labels):
    pad_token_id = tokenizer.pad_token_id
    decoder_input_ids = tf.concat(
        [tf.fill([tf.shape(labels)[0], 1], pad_token_id), labels[:, :-1]],
        axis=-1,
    )
    return decoder_input_ids

train_decoder_input_ids = shift_right(train_encodings["input_ids"])
val_decoder_input_ids = shift_right(val_encodings["input_ids"])


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_decoder_input_ids
}).shuffle(500).batch(8)

val_dataset = tf.data.Dataset.from_tensor_slices({
    "input_ids": val_encodings["input_ids"],
    "attention_mask": val_encodings["attention_mask"],
    "labels": val_decoder_input_ids
}).batch(8)

In [10]:
labels = np.array(labels)

In [11]:
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss=model.compute_loss)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [12]:
# tokenizer = T5Tokenizer.from_pretrained("t5-small")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer) 

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

In [ ]:
# model.save(r"C:\Users\jasmi\Projects\Final Project\PoliceTraining-Final\trained_model.keras")

In [ ]:
# Save the Hugging Face model
model.save_pretrained(r"C:\Users\jasmi\Projects\Final Project\PoliceTraining-Final\trained_model")

In [13]:
# Load the Hugging Face model
loaded_model = TFT5ForConditionalGeneration.from_pretrained(r"C:\Users\jasmi\Projects\Final Project\PoliceTraining-Final\trained_model")

# Verify the loaded model
loaded_model.summary()

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at C:\Users\jasmi\Projects\Final Project\PoliceTraining-Final\trained_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


Model: "tft5_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Reduce batch size for the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices({
    "input_ids": val_encodings["input_ids"],
    "attention_mask": val_encodings["attention_mask"],
    "labels": val_decoder_input_ids
}).batch(8)  # Reduce from 8 to 4

In [15]:
val_encodings = tokenizer(
    val_texts,
    padding="max_length",
    truncation=True,  # Truncate to fit max_length
    max_length=512,   # Ensure this matches your model's max input length
    return_tensors="tf"
)


In [16]:
with tf.device('/GPU:0'):  # Or '/TPU:0' if using TPU
    predictions = model.predict(val_dataset)

100/100 [==============================] - 199s 2s/step


ResourceExhaustedError: {{function_node __wrapped__ConcatV2_N_100_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[200,512,32128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:ConcatV2] name: concat

In [19]:
# predictions = model.predict(val_dataset)

# extract ze predicted labels from logits
predicted_logits = predictions.logits
predicted_labels = tf.argmax(predicted_logits, axis=-1)

NameError: name 'predictions' is not defined

In [17]:
# flatten true_labels to compare
true_labels = []
for batch in val_dataset:
    true_labels.extend(batch["labels"].numpy().flatten())

# change (don't ask, tell) true_labels to NumPy 
true_labels = np.array(true_labels)

In [18]:
# filter padding tokens get em out
pad_token_id = tokenizer.pad_token_id

valid_indices = true_labels != pad_token_id
filtered_true_labels = true_labels[valid_indices]
filtered_predicted_labels = predicted_labels.numpy().flatten()[valid_indices]

NameError: name 'predicted_labels' is not defined

In [ ]:
# Accuracy
accuracy = accuracy_score(filtered_true_labels, filtered_predicted_labels)

# Precision, Recall, and F1 Score
precision = precision_score(filtered_true_labels, filtered_predicted_labels, average="binary")
recall = recall_score(filtered_true_labels, filtered_predicted_labels, average="binary")
f1 = f1_score(filtered_true_labels, filtered_predicted_labels, average="binary")

# Confusion Matrix
conf_matrix = confusion_matrix(filtered_true_labels, filtered_predicted_labels)

# Display Results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
# plotting the confusion
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Success", "Failure"], yticklabels=["Success", "Failure"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     df["Bodycam Transcript"], 
#     df["Success/Failure"],     
#     test_size=0.2,            
#     random_state=42            
# )



# train_labels = train_labels.to_numpy().reshape(-1, 1)
# val_labels = val_labels.to_numpy().reshape(-1, 1)

In [ ]:
# # Split the data
# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     texts, labels, test_size=0.2, random_state=42
# )

# # Tokenize validation data
# val_encodings = tokenizer(
#     val_texts,
#     padding="max_length",
#     truncation=True,
#     max_length=max_length,
#     return_tensors="tf"
# )

# # Shift decoder input IDs for validation
# val_decoder_input_ids = shift_right(val_encodings["input_ids"])

# # Create TensorFlow datasets
# batch_size = 8  # Adjust batch size as needed

# train_dataset = tf.data.Dataset.from_tensor_slices({
#     "input_ids": train_encodings["input_ids"],
#     "attention_mask": train_encodings["attention_mask"],
#     "labels": train_decoder_input_ids
# }).shuffle(500).batch(batch_size)

# val_dataset = tf.data.Dataset.from_tensor_slices({
#     "input_ids": val_encodings["input_ids"],
#     "attention_mask": val_encodings["attention_mask"],
#     "labels": val_decoder_input_ids
# }).batch(batch_size)

In [ ]:
# train_labels_flat = train_labels.ravel()  
# classes = np.array([0, 1])  
# class_weights = compute_class_weight("balanced", classes=classes, y=train_labels_flat)
# class_weights_dict = {0: class_weights[0], 1: class_weights[1]}

In [ ]:
# sample_weights = []
# for label in train_labels_flat:  
#     weight = class_weights_dict[int(label)]  
#     sample_weights.append(weight)
#     print(f"Label: {label}, Assigned Weight: {weight}")

# # convert np.array
# sample_weights = np.array(sample_weights)

In [ ]:
# train_decoder_input_ids = shift_right(train_encodings["input_ids"])
# val_decoder_input_ids = shift_right(val_encodings["input_ids"])

# # create dataset
# train_dataset = tf.data.Dataset.from_tensor_slices({
#     "input_ids": train_encodings["input_ids"],
#     "attention_mask": train_encodings["attention_mask"],
#     "labels": train_decoder_input_ids
# }).shuffle(500).batch(8, drop_remainder=True) #shuffle 1000 or 500 / / batch 4,8 or 16

# val_dataset = tf.data.Dataset.from_tensor_slices({
#     "input_ids": val_encodings["input_ids"],
#     "attention_mask": val_encodings["attention_mask"],
#     "labels": val_decoder_input_ids
# }).batch(8)

In [ ]:
# dataset = tf.data.Dataset.from_tensor_slices((
#     {
#         "input_ids": input_encodings["input_ids"],
#         "attention_mask": input_encodings["attention_mask"],
#         "decoder_input_ids": label_encodings["input_ids"]  
#     },
#     label_encodings["input_ids"]  
# ))
# dataset = dataset.shuffle(1000).batch(8)

In [ ]:
# def custom_loss_function(labels, logits):
#     loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
#         from_logits=True, reduction=tf.keras.losses.Reduction.NONE
#     )
#     tf.print("Labels in Custom Loss Function:", labels)
#     mask = tf.cast(labels != 0, dtype=tf.float32)  
#     loss = loss_fn(labels, logits)
#     loss *= mask
#     weights = tf.gather([class_weights_dict[0], class_weights_dict[1]], labels)
#     loss *= weights  
#     return tf.reduce_mean(loss)

In [ ]:
# def custom_loss_function(labels, logits):
#     # bug it
#     tf.print("Labels Shape (before reshape):", tf.shape(labels))
#     tf.print("Logits Shape:", tf.shape(logits))

    
#     labels = tf.reshape(labels, [-1])  
#     logits = tf.reshape(logits, [-1, tf.shape(logits)[-1]])  

    
#     tf.print("Labels Shape (after reshape):", tf.shape(labels))
#     tf.print("Logits Shape (after reshape):", tf.shape(logits))

    
#     loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#     loss = loss_fn(labels, logits)

#     return tf.reduce_mean(loss)  





In [ ]:
# Custom loss function ignoring PAD tokens
def custom_loss_function(labels, logits):
    mask = tf.not_equal(labels, tokenizer.pad_token_id)  # Mask out padding tokens
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction="none")
    loss = loss_fn(labels, logits)
    loss = tf.reduce_mean(loss * tf.cast(mask, tf.float32))  # Apply mask
    return loss

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=custom_loss_function) 

In [ ]:
print("Class Weights Mapping:", class_weights_dict)

In [ ]:
print("Unique Train Labels:", set(train_labels.flatten().tolist()))
print("Unique Validation Labels:", set(val_labels.flatten().tolist()))

In [ ]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)

In [ ]:
# model.fit(dataset, epochs=3)

In [ ]:
# from sklearn.model_selection import train_test_split


# train_texts, val_texts, train_labels, val_labels = train_test_split(
#     df["Bodycam Transcript"],  
#     df["Success/Failure"],     
#     test_size=0.2,             
#     random_state=42
# )

In [ ]:
val_encodings = tokenizer(
    list(val_texts), truncation=True, padding="max_length", max_length=256, return_tensors="tf"
)

predictions = []
for i in range(len(val_texts)):
    input_ids = val_encodings["input_ids"][i:i+1]
    attention_mask = val_encodings["attention_mask"][i:i+1]
    output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=10)
    pred = tokenizer.decode(output[0], skip_special_tokens=True)
    predictions.append(pred.strip().lower())

In [ ]:
predicted_labels = [1 if pred == "success" else 0 for pred in predictions]

true_labels = list(val_labels)



In [ ]:
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=["Failure", "Success"]))

In [ ]:
train_class_distribution = train_labels.value_counts()
val_class_distribution = val_labels.value_counts()

print("Training Class Distribution:")
print(train_class_distribution)

print("\nValidation Class Distribution:")
print(val_class_distribution)

In [ ]:
!pip install wordcloud
from wordcloud import WordCloud, STOPWORDS

In [ ]:
custom_stopwords = set(STOPWORDS)
custom_stopwords.update(["officer", "miller", "subject", "sound", "october", "time", "and", "john", "doe", "sir", "I", "Audio", "Bodycam", "Transcript"])

text_data = " ".join(val_texts)
wordcloud = WordCloud(
    width=800, height=400, background_color="white", stopwords=custom_stopwords, colormap="viridis"
).generate(text_data)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Bodycam Transcripts (Excluding Common Words)", fontsize=16)
plt.show()